In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
files = sorted(glob('output-nusantara/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

150

In [5]:
import mp
import copy

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            results.append(data[i])

    return results

In [6]:
results = mp.multiprocessing(files, loop, cores = 50)

100%|██████████| 3/3 [00:00<00:00, 640.12it/s]




100%|██████████| 3/3 [00:00<00:00, 604.60it/s]

100%|██████████| 3/3 [00:00<00:00, 897.24it/s]

100%|██████████| 3/3 [00:00<00:00, 1156.09it/s]



100%|██████████| 3/3 [00:00<00:00, 935.12it/s]

100%|██████████| 3/3 [00:00<00:00, 992.26it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 927.60it/s]


100%|██████████| 3/3 [00:00<00:00, 1028.35it/s]






100%|██████████| 3/3 [00:00<00:00, 957.31it/s]





In [15]:
len(results)

2222052

In [16]:
results[0]

('output-audio/2-0-0.mp3',
 {'predict_ms': [50258,
   50282,
   50360,
   50365,
   4586,
   455,
   6801,
   991,
   25835,
   19377,
   2938,
   50409,
   50409,
   9118,
   25835,
   50434,
   50434,
   591,
   4118,
   3913,
   388,
   5581,
   1606,
   1508,
   9568,
   16434,
   50502,
   50502,
   10715,
   1508,
   9568,
   514,
   50539,
   50539,
   479,
   545,
   335,
   283,
   545,
   335,
   283,
   545,
   335,
   50597,
   50597,
   5707,
   335,
   2604,
   50638,
   50638,
   407,
   290,
   29319,
   5948,
   650,
   2394,
   50708,
   50708,
   460,
   29319,
   5948,
   650,
   2394,
   50790,
   50790,
   5130,
   2419,
   50803,
   50803,
   19158,
   296,
   12,
   7124,
   296,
   3867,
   50853,
   50853,
   5637,
   650,
   2394,
   50894,
   50894,
   33468,
   4714,
   9568,
   514,
   26532,
   50994,
   50994,
   508,
   1459,
   71,
   50996,
   50996,
   508,
   1459,
   71,
   312,
   2394,
   6801,
   51034,
   51034,
   879,
   2394,
   816,
   897,

In [7]:
with open('prepared-nusantara-original.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 6274/6274 [00:00<00:00, 35369.71it/s]


In [20]:
# !zip -r -s 10000M split-nusantara.zip split-nusantara

In [19]:
!ls split-nusantara.z*

split-nusantara.zip


In [21]:
tokenizer.decode(results[0]['predict_ms'])

'<|startoftranscript|><|ms|><|transcribe|> Bab 3 Orang Turk memperjuangkan hak mereka. Orang ramai lalu-lalang melintasi arca Hassan Tahsin yang berdiri di Medan Konak, iaitu bangunan kerajaan, yang terletak berhampiran dermaga Izmir yang tersergam indah dan baru siap dibina semula. Hassan Tahsin ialah nama samaran seorang wartawan yang telah menembak mati pembawa panji pasukan tentera<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [22]:
from huggingface_hub import HfApi
api = HfApi()

In [23]:
api.upload_file(
    path_or_fileobj='prepared-nusantara-original.jsonl',
    path_in_repo='prepared-nusantara-original.jsonl',
    repo_id='mesolitica/pseudolabel-nusantara-large-v3-timestamp',
    repo_type='dataset',
)

prepared-nusantara-original.jsonl:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-nusantara-large-v3-timestamp/blob/main/prepared-nusantara-original.jsonl'

In [24]:
files = glob('split-nusantara.z*')
for f in files:
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/pseudolabel-nusantara-large-v3-timestamp',
        repo_type='dataset',
    )

split-nusantara.zip


split-nusantara.zip:   0%|          | 0.00/911M [00:00<?, ?B/s]